In [ ]:
import os
import random
import gym
import pylab
import numpy as np
from collections import deque
from keras.models import Model, load_model
from keras.layers import Input, Dense, Lambda, Add
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.callbacks import TensorBoard
import time

In [ ]:
NAME = "D3QN_Egreedy_Cartpole-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(NAME))

In [ ]:
def RLModel(input_shape, action_space):
    X_i = Input(input_shape)
    X = X_i
    X = Dense(512, input_shape=input_shape, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(256, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)

    state_value = Dense(1, kernel_initializer='he_uniform')(X)
    state_value = Lambda(lambda s: K.expand_dims(s[:, 0], -1), output_shape=(action_space,))(state_value)

    action_advantage = Dense(action_space, kernel_initializer='he_uniform')(X)
    action_advantage = Lambda(lambda a: a[:, :] - K.mean(a[:, :], keepdims=True), output_shape=(action_space,))(action_advantage)

    X = Add()([state_value, action_advantage])
    
    model = Model(inputs = X_i, outputs = X, name='CartPole D3QN_Egreedy model')
    model.compile(loss="mean_squared_error", optimizer=RMSprop(lr=0.0025, rho=0.95, epsilon=0.01), metrics=["accuracy"])

    model.summary()
    return model

In [ ]:
class DQNAgent:
    def __init__(self, env_name):
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.env.seed(0)  
        self.env._max_episode_steps = 4000
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n

        self.EPISODES = 500
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate

        self.epsilon = 1.0 
        self.epsilon_min = 0.01 
        self.epsilon_decay = 0.999
        
        self.batch_size = 32

        self.TAU = 0.1 

        self.Save_Path = 'Models'
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.scores, self.episodes, self.average = [], [], []

        self.Model_name = os.path.join(self.Save_Path, self.env_name+"_e_greedy.h5")
        
        self.model = RLModel(input_shape=(self.state_size,), action_space = self.action_size)
        self.target_model = RLModel(input_shape=(self.state_size,), action_space = self.action_size)

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        return
        
    def memory_fun(self, state, action, reward, next_state, done):
        experience = state, action, reward, next_state, done
        self.memory.append((experience))

    def act(self, state, decay_step):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= (1-self.epsilon_decay)
        
        explore_probability = self.epsilon

        if explore_probability > np.random.rand():
            return random.randrange(self.action_size), explore_probability
        else
            return np.argmax(self.model.predict(state)), explore_probability

    def replay_buffer(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        target = self.model.predict(state)
        target_next = self.model.predict(next_state)
        target_val = self.target_model.predict(next_state)

        for i in range(len(minibatch)):
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                a = np.argmax(target_next[i])
                target[i][action[i]] = reward[i] + self.gamma * (target_val[i][a])   
                
        self.model.fit(state, target, batch_size=self.batch_size, verbose=0, callbacks=[tensorboard])

    def loadModel(self, name):
        self.model = load_model(name)

    def saveModel(self, name):
        self.model.save(name)

    pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        pylab.plot(self.episodes, self.average, 'r')
        pylab.plot(self.episodes, self.scores, 'b')
        pylab.ylabel('Score', fontsize=18)
        pylab.xlabel('Steps', fontsize=18)
        
        try:
            pylab.savefig("D3QN_egreedy_carpole.png")
        except OSError:
            pass

        return str(self.average[-1])[:5]
    
    def trainModel(self):
        decay_step = 0
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                #self.env.render()
                decay_step += 1
                action, explore_probability = self.act(state, decay_step)
                next_state, reward, done, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])
                if not done or i == self.env._max_episode_steps-1:
                    reward = reward
                else:
                    reward = -100
                self.memory_fun(state, action, reward, next_state, done)
                state = next_state
                i += 1
                if done:
                    self.update_target_model()
                    average = self.PlotModel(i, e)
                    print("episode: {}/{}, score: {}, e: {:.2}, average: {}".format(e, self.EPISODES, i, explore_probability, average))
                    if i == self.env._max_episode_steps:
                        self.saveModel(self.Model_name)
                        self.env.close()
                        return

                self.replay_buffer()

    def testModel(self):
        self.loadModel(self.Model_name)
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
                self.env.render()
                action = np.argmax(self.model.predict(state))
                next_state, reward, done, _ = self.env.step(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += 1
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, i))
                    break

In [ ]:
if __name__ == "__main__":
    env_name = 'CartPole-v1'
    agent = DQNAgent(env_name)
    agent.trainModel()
    #agent.testModel()